**Initial Test Accuracy:** 94,5%<br>
**Final Test Accuracy:** 99,5%<br>
**Adjustments:**<br>
*   max_words = 400 -> 4000
*   max_len = 5 -> 10
*   Dropout added
*   Learning Rate = 0.001 -> 0.01

In [203]:
# !pip install stop_words pymorphy2

In [204]:
from torch import nn
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import random
import torch


In [205]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [206]:
max_words = 4000
max_len = 10
num_classes = 1

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

th = 0.5

In [207]:
import pandas as pd

df_train = pd.read_csv("train.csv")
final_test = pd.read_csv("test.csv")


In [208]:
X_train, X_test, y_train, y_test = train_test_split(df_train['tweet'], df_train['label'], test_size=0.25, random_state=26)

In [209]:
X_train.head(2)

10549    @user so sweet. .  fathers day ..love u devð...
24767                              porn vids web free sex 
Name: tweet, dtype: object

In [210]:
X_test.head(2)

22757    visiting a friend and finding your book in the...
7670     can't help #smiling when i #color this!   #sat...
Name: tweet, dtype: object

In [211]:
sw = set(get_stop_words("en"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\snot", "not", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [212]:
X_train.head(2)

10549    user sweet fathers day love u devðððð...
24767                               porn vids web free sex
Name: tweet, dtype: object

In [213]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [214]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [215]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [216]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [217]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'u', 'time', 'im']

In [218]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [219]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [220]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [221]:
x_train[12345]

array([   0,    0, 3678, 3965,  338,   25,  122,  750, 2158, 1954],
      dtype=int32)

In [222]:
seed = 26

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [223]:
from torch.utils.data import DataLoader, Dataset

class DataWrapper(Dataset):
    def __init__(self, data, target=None, transform=None):
        self.data = torch.from_numpy(data).long()
        if target is not None:
            self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index] if self.target is not None else None

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [224]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True) :
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        gru_out, ht = self.gru(x)

        if self.use_last:
          last_tensor = gru_out[:,-1,:]
        else:
          last_tensor = torch.mean(gru_out[:,:], dim=1)

        out = self.linear(last_tensor)
        return torch.sigmoid(out)


In [225]:
gru_init = GRUFixedLen(max_words, 128, 20, use_last=True).to(device)
optimizer = torch.optim.Adam(gru_init.parameters(), lr=0.01)
criterion = nn.BCELoss()

print(gru_init)
print("Parameters:", sum([param.nelement() for param in gru_init.parameters()]))


GRUFixedLen(
  (embeddings): Embedding(4000, 128, padding_idx=0)
  (gru): GRU(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 523541


In [226]:
gru_init.train()
#model = model.cuda()

train_dataset = DataWrapper(x_train, y_train.values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DataWrapper(x_test, y_test.values)
test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [227]:

manual_acc = []
for epoch in tqdm(range(epochs)):
    gru_init.train()
    right, num = 0, 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = gru_init(inputs)

        loss = criterion(outputs, labels.float().view(-1,1))
        loss.backward()
        optimizer.step()
        right += (labels == torch.squeeze((outputs > th).int())).sum()
        num += len(labels)

    gru_init.eval()
    loss_accumed, test_right, test_num = 0, 0, 0
    for i, data in enumerate(test_loader, 0):
        X, y = data[0].to(device), data[1].to(device)
        output = gru_init(X)
        loss = criterion(output, y.float().view(-1,1))
        loss_accumed += loss
        test_num += len(y)
        test_right += (y == torch.squeeze((output > th).int())).sum()

    train_acc = right/num
    test_acc = test_right/test_num
    manual_acc.append(test_acc)
    print(f"Epoch {epoch+1}; test_loss {loss_accumed:.3f}; test_acc {test_acc:.3f}; train_acc {train_acc:.3f}")

print('Training is finished!')

 10%|█         | 1/10 [00:00<00:04,  2.03it/s]

Epoch 1; test_loss 7.577; test_acc 0.950; train_acc 0.932


 20%|██        | 2/10 [00:01<00:04,  1.96it/s]

Epoch 2; test_loss 4.666; test_acc 0.969; train_acc 0.955


 30%|███       | 3/10 [00:01<00:03,  1.99it/s]

...

Epoch 8; test_loss 0.872; test_acc 0.994; train_acc 0.986


 90%|█████████ | 9/10 [00:04<00:00,  1.86it/s]

Epoch 9; test_loss 0.765; test_acc 0.995; train_acc 0.988


100%|██████████| 10/10 [00:05<00:00,  1.88it/s]

Epoch 10; test_loss 0.702; test_acc 0.995; train_acc 0.989
Training is finished!
